### KnowEnG-Research/Data_Cleanup_Pipeline test all methods notebook (for script run on cloud)
    Use this notebook to test the KnowEnG... /Data_Cleanup_Pipeline Locally:


* Create a directory to hold all this code with the test tools repository cloned into it.
    * call it homely_directory
    * git clone https://github.com/dlanier/keg_test_tools.git


* Clone the Data_Cleanup_Pipeline from github. 
    * git clone https://github.com/KnowEnG/Data_Cleanup_Pipeline.git
    * change to Data_Cleanup_Pipeline/test and run >make env_setup
    * change back to the homely_directory


* Clone the Samples_Clustering_Pipeline from github. 
    * git clone https://github.com/KnowEnG/Data_Cleanup_Pipeline.git
    * change to Samples_Clustering_Pipeline/test and run >make env_setup
    * change back to the homely_directory


* Clone the Clustering_Eval_Pipeline from github. 
    * git clone https://github.com/KnowEnG/Data_Cleanup_Pipeline.git
    * change to Clustering_Eval_Pipeline/test and run >make env_setup
    
    
* Run this jupyter notebook

In [26]:
SHOW_COMMAND_LINE_OUTPUT = True

import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time

import pandas as pd
import numpy as np

dcp_src = '../../Data_Cleanup_Pipeline/src/'
sys.path.insert(1, dcp_src)
import data_cleanup_toolbox as dc

scp_src = '../../Samples_Clustering_Pipeline/src/'
sys.path.insert(1, scp_src)
import sample_clustering_toolbox as sc

cep_src = '../../Cluster_Evaluation_Pipeline/src'
sys.path.insert(1, cep_src)
import clustering_eval_toolbox as ce

sys.path.insert(1, '../../keg_test_tools/src')
import dcp_test
import samples_clustering_test_module as sc_test

import knpackage.toolbox as kn

# If not running inside a docker container of the KnowEnG variety: Create this Directory
output_data_dir = '../../test/run_dir'
yaml_dir = '../../Data_Cleanup_Pipeline/data/run_files'
yaml_file = 'TEMPLATE_data_cleanup.yml'

In [60]:
import samples_clustering_test_module as sc_test

In [27]:
run_parameters = kn.get_run_parameters(yaml_dir, yaml_file)

#                   Data_Cleanup_Pipeline   Samples Clustering
data_root = '/Users/lanier4/pipeline_spreadsheets/aws_v3'

sc_phenotypes_path = os.path.join(data_root, 'SC_P_spreadsheets')
run_parameters['pheno_data_dir'] = sc_phenotypes_path
run_parameters['phenotype_name_full_path'] = os.path.join(
        run_parameters['pheno_data_dir'], 'Hsap.ccle.P.cell_line_info.mixed.c.df')

sc_spreadsheets_path = os.path.join(data_root, 'SC_G_spreadsheets')
run_parameters['spreadsheet_data_dir'] = sc_spreadsheets_path
run_parameters['spreadsheet_name_full_path'] = os.path.join(
        run_parameters['spreadsheet_data_dir'], 'Hsap.ccle.G.gene_mut.binary.a.df')

run_parameters['pipeline_type'] = 'samples_clustering_pipeline'

run_parameters['run_directory'] = dcp_src
run_parameters['results_directory'] = '../test/tmp'

run_parameters['SHOW_COMMAND_LINE_OUTPUT'] = True
if SHOW_COMMAND_LINE_OUTPUT:
    for p in list(run_parameters.keys()):
        print(p, ':\t\t', run_parameters[p])

# test_result_df = dcp_test.test_data_cleanup(run_parameters)

spreadsheet_name_full_path :		 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets/Hsap.ccle.G.gene_mut.binary.a.df
pheno_data_dir :		 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets
run_directory :		 ../../Data_Cleanup_Pipeline/src/
results_directory :		 ./
SHOW_COMMAND_LINE_OUTPUT :		 True
source_hint :		 
pipeline_type :		 samples_clustering_pipeline
spreadsheet_data_dir :		 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets
taxonid :		 9606
phenotype_name_full_path :		 ../data/spreadsheets/TEST_1_phenotype.tsv
phenotype_full_path :		 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets/Hsap.ccle.P.cell_line_info.mixed.c.df
redis_credential :		 {'host': 'knowredis.knowhub.org', 'port': 6379, 'password': 'KnowEnG'}
run_file :		 TEMPLATE_data_cleanup.yml


In [ ]:
spreadsheet_data_dir = run_parameters['spreadsheet_data_dir']
pheno_data_dir = run_parameters['pheno_data_dir']

t0 = time.time()
sc_test_list_df = sc_test.get_SC_phenotype_viability_df(spreadsheet_data_dir, pheno_data_dir)
tt = time.time() - t0
print('Run time: %0.3f'%tt)

In [ ]:
# sc_test_list_df
row_list = list(sc_test_list_df.index)
row_n = row_list[0]
spreadsheet_name = sc_test_list_df['spreadsheet'].loc[row_n]
phenotype_name = sc_test_list_df['phenotype'].loc[row_n]

In [ ]:
run_parameters['results_directory'] = '../test/tmp'

for row_n in list(sc_test_list_df.index):
    if sc_test_list_df['samples_intersect'].loc[row_n] > 0:
        spreadsheet_name = sc_test_list_df['spreadsheet'].loc[row_n]
        run_parameters['spreadsheet_name_full_path'] = os.path.join(
            run_parameters['spreadsheet_data_dir'], spreadsheet_name)
        
        phenotype_name = sc_test_list_df['phenotype'].loc[row_n]
        run_parameters['phenotype_name_full_path'] = os.path.join(
            run_parameters['pheno_data_dir'], phenotype_name)
        
        validation_flag, message = dc.run_samples_clustering_pipeline(run_parameters)
        
        if validation_flag:
            print('Success! = ', validation_flag)
            print(run_parameters['spreadsheet_name_full_path'], '\n\t', run_parameters['phenotype_full_path'])
#             print(message)
        else:
            print('Fail ! = ', validation_flag)
            print(run_parameters['spreadsheet_name_full_path'], '\n\t', run_parameters['phenotype_full_path'])
#             print(message)

Success! =  True
/Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets/Hsap.ccle.G.gene_mut.binary.a.df 
	 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets/Hsap.tcga_ucsc.P.clinical.mixed.c.df
Success! =  True
/Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets/Hsap.ccle.G.gene_mut.binary.a.df 
	 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets/Hsap.tcga_ucsc.P.clinical.mixed.c.df
Success! =  True
/Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets/Hsap.ccle.G.gene_mut.binary.a.df 
	 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets/Hsap.tcga_ucsc.P.clinical.mixed.c.df
